In [1]:
import os, sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.io 
import random
from networks.DMGP import*
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1" # Will use only the first and the second GPU devices

np.random.seed(0)
tf.set_random_seed(0)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f = open('./JURA/jura.txt', 'r')
D = []
for line in f:
    D.append(line.split())
D = np.asarray(D).astype(np.float)
f.close()

Xtrain_raw = D[:,0:2]
Ytrain_raw = D[:,4:]

Xmean = np.mean(Xtrain_raw, axis=0, keepdims=True)
Xstd = np.std(Xtrain_raw, axis=0, keepdims=True)
Xtrain = (Xtrain_raw - Xmean) / Xstd

Ymean = np.mean(Ytrain_raw, axis=0, keepdims=True)
Ystd = np.max(np.std(Ytrain_raw, axis=0, keepdims=True))
Ytrain = (Ytrain_raw - Ymean) / Ystd

#====================================================================

f = open('./JURA/jura_test.txt', 'r')
D = []
for line in f:
    D.append(line.split())
D = np.asarray(D).astype(np.float)
f.close()

Xtest_raw = D[:,0:2]
Ytest_raw = D[:,4:]

Xtest = (Xtest_raw - Xmean) / Xstd
Ytest = (Ytest_raw - Ymean) / Ystd
Dout = 7

In [3]:
# tosee = [0,4,6]
# Ytrain_raw = Ytrain_raw[:,tosee]
# Ytest_raw = Ytest_raw[:,tosee]
# Ytrain = Ytrain[:,tosee]
# Ytest = Ytest[:,tosee]
# Ymean = Ymean[:,tosee]
# Ystd = Ystd[:,tosee]

# print(Ymean.shape, Ystd.shape)
# print(Xtrain.shape, Ytrain.shape)
# print(Xtest.shape, Ytest.shape)
# Dout = len(tosee)

In [4]:
# Define DMGP parameters
N_LAYER = 2 # the number of layers
D = [2, Dout, Dout] # dimensions for each layer
M = [20, 20] # the number of pseudo-inputs for each layer

# N_LAYER = 1 # the number of layers
# D = [2, Dout] # dimensions for each layer
# M = [40] # the number of pseudo-inputs for each layer

K = 20 # the number of samples

DGP_net = DGP_DVSI(N_LAYER, D, M, K, Mat=False, MCO=False)

In [5]:
import time
t1 = time.time()
elbo_hist, elbo_hist_test = DGP_net.train(Xtrain,Ytrain,Xtest,Ytest,epoch=1000)
print (time.time() - t1)

0 20678.004 -20678.004
100 2836.6284 -2836.6284
200 2013.0016 -2013.0016
300 1392.5068 -1392.5068
400 963.70245 -963.70245
500 637.66724 -637.66724
600 449.50308 -449.50308
700 365.14294 -365.14294
800 311.72736 -311.72736
900 313.7426 -313.7426
76.83666276931763


In [6]:
Yest_mean, Yest_var = DGP_net.predict(Xtest, K=1000)
Yest_ll = -0.5*(Yest_mean - Ytest)**2 / Yest_var - 0.5*np.log(2*np.pi) - 0.5*np.log(Yest_var)

print(max(elbo_hist))
print(np.mean(np.sqrt(np.sum((Yest_mean - Ytest)**2,axis=1)), axis=0))
# print(np.mean(Yest_ll, axis=0))
print(np.sum(Yest_ll))
#print(np.mean(Yest_var, axis=0))

-255.87354
1.3192126593917812
-168.56188646809488


In [7]:
# dmgp
[0.01998158 0.07596444 0.23610052 0.63558149 0.16316    0.85245612
 0.77979153]
[ 2.25810577  0.96356146 -0.22476392 -1.18875558  0.14632828 -1.70179153
 -1.49222733]
-123.95428594453783
[7.7863497e-04 7.9584047e-03 1.1084138e-01 6.6802812e-01 4.7088277e-02
 1.3774719e+00 9.5316470e-01]

# mgp
[0.01916131 0.07579337 0.24587666 0.56432743 0.16831863 0.77354327
 0.75560861]
[ 2.25231647  0.92643309 -0.20386651 -1.18084596  0.15570775 -1.77655423
 -1.49219067]
-131.90000686463083
[7.93336716e-04 7.63225090e-03 1.17920466e-01 5.26437342e-01
 5.04407510e-02 1.01887715e+00 8.25399399e-01]

# dgp
[0.0195901  0.08121365 0.23071248 0.58180515 0.16842543 0.79107847
 0.76241053]
[ 1.30964188  0.85058792 -0.17783508 -1.35025023  0.14225919 -1.83247777
 -1.69566264]
-275.37367176453097
[0.01095944 0.0148313  0.10183809 0.47026053 0.04551153 0.9540388
 0.72243536]

# gp


SyntaxError: invalid syntax (<ipython-input-7-a295bde1ab88>, line 2)